In [6]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wd
from selenium.webdriver.common.by import By

# 명시적 대기를 위한 package
from selenium.webdriver.support.ui import WebDriverWait
# 암묵적 대기를 위한 package
from selenium.webdriver.support import expected_conditions as ec

import time
from datetime import datetime

import re

from exercise_Selenium_Tour import TourInfo

In [7]:
driver_path = 'C:/PythonHome/chromedriver.exe'

url = 'http://tour.interpark.com/'
keyword = '미국여행'
tour_list = []

driver = wd.Chrome(executable_path=driver_path)
driver.get(url)
# 검색창에 keyword 입력 - id: SerchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
# 검색 - class: search-btn
# driver.find_element_by_class_name('search-btn').click()
# 검색 - class: search-btn, type: button
# Class를 접근할 때는 .으로 구분, ID로 접근할 때는 #으로 구분
driver.find_element_by_css_selector('button.search-btn').click()

# 명시적 대기: 특정 element가 load될 때 까지 대기
# 암묵적 대기: 일정 시간 대기

now = datetime.now()
# 명시적 대기
try:
    #print('명시적 대기 전: {}:{}:{}'.format(now.hour, now.minute, now.second))
    
    WebDriverWait(driver, 10).until(
        # 지정한 요소가 로딩될 때 까지 대기
        # <div class = 'oTravelBox'..>
        ec.presence_of_element_located((By.CLASS_NAME, 'oTravelBox'))
    )
    #print('명시적 대기 후: {}:{}:{}'.format(now.hour, now.minute, now.second))
except Exception as e:
    print('Exception Occurrance, reason:', e)

# 암묵적 대기
#print('암묵적 대기 전: {}:{}:{}'.format(now.hour, now.minute, now.second))
driver.implicitly_wait(10)
#print('암묵적 대기 후: {}:{}:{}'.format(now.hour, now.minute, now.second))
# time.sleep(10)

driver.find_element_by_css_selector(
    '.oTravelBox > .boxList > .moreBtnWrap > .moreBtn').click()

# 상품명, comment, 가격, 평점, 이미지 ,링크(상품상세정보), 기간1, 기간2
# proTit, proSub, proPrice, proInfo, img, onClick의 주소, proInfo, box
# <li class='boxItem'
try:
    time.sleep(2)
    boxItems = driver.find_elements_by_css_selector(
        '.oTravelBox > .boxList > .boxItem'
    )

    for box in boxItems:
        title = box.find_element_by_css_selector('h5.proTit').text
        
        price = box.find_element_by_css_selector('strong.proPrice').text
        price = int(re.sub(r'[+, 원~+]', '', price))
        
        comment = box.find_element_by_css_selector('p.proSub').text
        
        review_score = box.find_elements_by_css_selector('p.proInfo')[2].text
        review_score = float(review_score.split(' ')[1])
        
        travel = box.find_elements_by_css_selector('p.proInfo')[0].text
        p = re.compile('[0-9]+박[0-9]+일')
        travel_period = p.findall(travel)[0]
        
        transfer = travel.split(travel_period)[1]
        
        departure_period = box.find_elements_by_css_selector('p.proInfo')[1].text
        departure_period = departure_period.split(' : ')[1]
        
        tour_obj = TourInfo(
            title, price, comment, review_score, travel_period, departure_period
        )
        
        tour_list.append(tour_obj)
except Exception as e:
    print('Exception Occurrance, reason:', e)
finally:
    driver.close()

In [8]:
import pymongo
from pymongo import MongoClient

In [10]:
client = MongoClient('localhost', 27017)
db = client.test

for tour in tour_list:
    db.info.insert({
        'title': tour.title, 
        'price': tour.price, 
        'comment': tour.comment, 
        'review_score': tour.review_score, 
        'travel_period': tour.travel_period, 
        'departure_peroid': tour.departure_period
    })

C:\Users\Hyeonju\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # This is added back by InteractiveShellApp.init_path()
